In [1]:
%matplotlib inline 
import itertools
from bs4 import BeautifulSoup
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import matplotlib.pyplot as plt
from pyquery import PyQuery as pq
dropbox = "C:\Users\mkkes_000\Dropbox\Indiastuff\OutputTables"

In [150]:
list_elections = ["andhra2014","arunachal2014","bihar2015","chhattisgarh2013","delhi2015","gujarat2012","haryana2014","hp2012","jk2014",
              "jharkhand2014","karnataka2013","kerala2011","mp2013","maharashtra2014","manipur2012","odisha2014","pb2012",
              "rajasthan2013","sikkim2014","tamilnadu2011","telangana2014","tripura2013","utt2012","up2012","westbengal2011"]

In [151]:
def only_highest(l):
    list_highest = []
    for i in range(0,len(l)-1):
        if l[i+1]>l[i]:
            list_highest = list_highest
        else:
            list_highest.append(l[i])
    return list_highest

In [152]:
def consistuency_tuple(e):
    long_link = "http://myneta.info/"+e+"/"
    source=requests.get(long_link)
    time.sleep(1)
    tree= BeautifulSoup(source.text,"html.parser")
    names_districts = [x.get_text().strip().title() for x in tree.findAll("h5", { "class" : "title" })]
    list_const = tree.findAll("div", { "class" : "items" })[1:]
    names_consistuencies = [x.get_text().split(":")[1].strip().title().split("*")[0] for x in list_const]
    link_consist = [x.find("a").get("href") for x in tree.findAll("div", { "class" : "items" })[1:]]
    id_consist = [x.split("=")[2] for x in link_consist]
    # Only taking the first election
    n_consistuencies = only_highest([int(x.get_text().split(":")[0].split("(")[0])
                                 for x in tree.findAll("div", { "class" : "items" })[1:]])
    list_districts = list(itertools.chain(
                    *[[names_districts[i]]*n_consistuencies[i] for i in range(0,len(names_districts)-1)]))
    return zip(names_consistuencies,list_districts,id_consist)

In [153]:
%%time
#Creates a dictionary state:[all consistuencies] 
#where all consistuency is a tuple (consistuency,district,id of consistuency)
all_elections = {el:consistuency_tuple(el) for el in list_elections}

Wall time: 52.6 s


In [154]:
#just building dictionaries to store info more efficiently
def store_dict(el):
    dtuple = all_elections[el]
    consistuency_district =  {l[2]:l[1] for l in dtuple}
    return consistuency_district

def store_dict2(el):
    dtuple = all_elections[el]
    consistuency_code =  {l[2]:l[0] for l in dtuple}
    return consistuency_code

def store_consistuencies(el):
    dtuple = all_elections[el]
    allcodes =  [l[2] for l in dtuple]
    return allcodes

election_consistuencies = {}
allconst_dist = {}
allconst_codes = {}
for el in list_elections:
    allconst_dist[el] = store_dict(el)
    allconst_codes[el] = store_dict2(el)
    election_consistuencies[el] = store_consistuencies(el)

with open("tempdata/const_district",'w') as js_file:
    json.dump(allconst_dist,js_file)
with open("tempdata/const_codes",'w') as js_file:
    json.dump(allconst_codes,js_file)

In [155]:
# The cand_info function is at the consistuency level
# It takes a table of all candidates in the consistuency and returns a dictionary
# with all the info on each candidate

def cand_info(line):
    all_info = line.findAll("td")
    name = all_info[0].get_text()
    cand_link = all_info[0].find("a").get("href")
    cand_id = cand_link.split("=")[1]
    try:
        age = int(all_info[1].get_text())
    except TypeError:
        age = np.nan
    sex = all_info[2].get_text()
    party = all_info[3].get_text()
    serious_crim_yn = all_info[4].get_text()
    try:
        crim = int(all_info[5].get_text())
        serious_crim_num = int(all_info[6].get_text())
    except ValueError:
        crim = np.nan
        serious_crim_num = np.nan
    if all_info[7].get_text()=="":
        crim_types = []
    else:
        crim_types = [x.get_text() for x in all_info[7].findAll("span")]
    educ = all_info[8].get_text()
    list_assets = [int(all_info[i].get_text().split("~")[0].replace(',', '')) for i in [9,10,11,12]]
    #mov_assets = all_info[9].get_text().split("~")[0]
    #immov_assets = all_info[10].get_text().split("~")[0]
    #total_assets = all_info[11].get_text().split("~")[0]
    #total_liab = all_info[12].get_text().split("~")[0]
    pan = all_info[13].get_text()
    list_all = [cand_link,cand_id,name,age,sex,party,serious_crim_yn,crim,serious_crim_num,crim_types,educ]+list_assets+[pan]
    return list_all

In [149]:
#dict_all_pages = {}
del dict_all_pages['up2012']

In [156]:
# Creates a dictionary of dictionaries in the format:
# {election:{constituency_id:detailed consistuency page}}
def get_page(c,election):
    # Check if URL has already been visited.
    url_error = []
    if (c not in urlcache) or (urlcache[c]==1) or (urlcache[c]==2):
        time.sleep(.5)
        steps = len(urlcache)
        if 100*int(steps/100)==steps:
            print steps # This counter tells us how many links were downloaded at every 100 mark
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            link_disc = "http://myneta.info/"+election+"/comparisonchart.php?constituency_id="+c
            c_page=requests.get(link_disc)
            if c_page.status_code == 200:
                urlcache[c] = c_page.text
            else:
                urlcache[c] = 1
        except:
            urlcache[c] = 2
            url_error.append(c)
            print c
    return urlcache[c]

def get_info_consistuency(election):
    const_list = election_consistuencies[election]
    print "\n", election, " of length", len(const_list), ":",
    dict_store = {}
    for c in const_list:
        link_disc = "http://myneta.info/"+election+"/comparisonchart.php?constituency_id="+c
        time.sleep(.5)
        c_page=requests.get(link_disc)
        dict_store[c] = c_page
        if len(dict_store)%100==0 or len(dict_store)==len(const_list):
            print len(dict_store),
    return dict_store

In [157]:
%%time
# The structure is the following:
# List of dictionaries {state:{consistuency:page}}
for el in list_elections:
    if el in dict_all_pages:
        if len(dict_all_pages[el]) == len(election_consistuencies[el]):
            print "done for:", el, "length:", len(dict_all_pages[el])
        else:
            print "starting again for ", el
    else:
        dict_all_pages[el] = get_info_consistuency(el) #careful, very long call

done for: andhra2014 length: 160
done for: arunachal2014 length: 54
done for: bihar2015 length: 234
done for: chhattisgarh2013 length: 87
done for: delhi2015 length: 62
done for: gujarat2012 length: 177
done for: haryana2014 length: 86
done for: hp2012 length: 64
done for: jk2014 length: 84
done for: jharkhand2014 length: 77
done for: karnataka2013 length: 220
done for: kerala2011 length: 138
done for: mp2013 length: 225
done for: maharashtra2014 length: 281
done for: manipur2012 length: 57
done for: odisha2014 length: 140

pb2012  of length 114 : 100 114 done for: rajasthan2013 length: 192

sikkim2014  of length 24 : 24 done for: tamilnadu2011 length: 232
done for: telangana2014 length: 107
done for: tripura2013 length: 46
done for: utt2012 length: 67

up2012  of length 396 : 100 200 300 396 done for: westbengal2011 length: 285
Wall time: 7min 21s


In [165]:
#The all_const function is at the constituency level
# Taking a consistuency in a given state, it spits out a list of candidates with their info
def all_candidates(c,election):
    dict_c = dict_all_pages[election]
    const_page = dict_c[c]
    tree = BeautifulSoup(const_page.text,"html.parser")
    cand_lines = tree.findAll("tr")[1:]
    consistuency = allconst_codes[election][c]
    district = allconst_dist[election][c]
    titles = ["election","c","consistuency","district","link","id"]+[x.get_text() for x in tree.findAll("th")]
    info_candidates = [dict(zip(titles,[election,c,consistuency,district]+cand_info(r))) for r in cand_lines]
    return info_candidates

In [166]:
def state_mapper(el):
    all_cand_state = [all_candidates(c_line[2],el) for c_line in all_elections[el]]
    return all_cand_state

In [167]:
%%time
all_elections_dict = {}
for e in list_elections:
    all_elections_dict[e]=state_mapper(e)
    print e,
    all_elections_final = all_elections_dict.copy()

andhra2014 arunachal2014 bihar2015 chhattisgarh2013 delhi2015 gujarat2012 haryana2014 hp2012 jk2014 jharkhand2014 karnataka2013 kerala2011 mp2013 maharashtra2014 manipur2012 odisha2014 pb2012 rajasthan2013 sikkim2014 tamilnadu2011 telangana2014 tripura2013 utt2012 up2012 westbengal2011Wall time: 1min 24s



In [168]:
all_elections_final['up2012'][1]

[{u'Age': 27,
  u'Criminal Cases': u'No',
  u'Education Level': u'Graduate',
  u'Gender': u'M',
  u'IPC Details*': [],
  u'Immovable Assets': 151964000,
  u'Movable Assets': 3383804,
  u'Name': u'Abhinav Sharma',
  u'Number of Cases': 0,
  u'PAN Given(Y or N)': u'Y',
  u'Party Code': u'SP',
  u'Serious IPC Counts': 0,
  u'Total Assets': 155347804,
  u'Total Liabilities': 38267728,
  'c': u'89',
  'consistuency': u'Agra North ',
  'district': u'Agra',
  'election': 'up2012',
  'id': u'1604',
  'link': u'http://myneta.info/up2012/candidate.php?candidate_id=1604'},
 {u'Age': 25,
  u'Criminal Cases': u'Yes',
  u'Education Level': u'Post Graduate',
  u'Gender': u'M',
  u'IPC Details*': [u' rioting(147)',
   u' voluntarily causing hurt(323)',
   u' assault or criminal force otherwise than on grave provocation(352)',
   u'Intentional insult with intent to provoke breach of the peace(504)'],
  u'Immovable Assets': 0,
  u'Movable Assets': 67060,
  u'Name': u'Amit Kumar Diwakar ',
  u'Number of 

In [169]:
## Takes one of the elements of the all_election_final dictionary
## And spits out a dataframe with all the columns. We can then stack them
## at a later stage
def reducer(el):
    list_const = all_elections_final[el]
    large_list = []
    for cons in list_const:
        large_list.extend(cons)
    df = pd.DataFrame(large_list)
    return df

In [215]:
df_all = pd.DataFrame()
for el in list_elections:
    df = reducer(el)
    df_all = df_all.append(df)
df_fin = df_all.copy()

In [216]:
%%time
## finding winners ##
winners_dict = {}
for el in list_elections:
    link_winners = "http://myneta.info/"+el+"/index.php?action=summary&subAction=winner_analyzed&sort=candidate#summary"
    wins = requests.get(link_winners)
    time.sleep(1)
    tree = BeautifulSoup(wins.text,"html.parser")
    tables = tree.findAll("table")
    list_ids = [x.find("a").get("href").split("=")[1] for x in tables[2].findAll("tr")[2:]]
    winners_dict[el]=list_ids

In [ ]:
%%time
## finding repeat candidates ##
repeat_dict = {}
for el in list_elections:
    link_repeat = "http://myneta.info/"+el+"/index.php?action=recontestAssetsComparison"
    page = requests.get(link_repeat)
    time.sleep(1)
    tree = BeautifulSoup(page.text,"html.parser")
    tables = tree.findAll("table")
    list_titles = tables[2].findAll("th")
                                ef").split("=")[1] for x in tables[2].findAll("tr")[2:]]
    winners_dict[el]=list_ids

In [303]:
def comparison_clean(line,el):
    id2 = line.find("a").get("href").split("=")[3].split("&")[0] #id2 is the latest
    # We are going to match it to the df and add the new id + old info
    id1 = line.find("a").get("href").split("=")[4]
    elements = [x.get_text() for x in line.findAll("td")]
    name = elements[1].split("(")[0].strip()
    new_assets = elements[2].split(u'\xa0')[0].replace(",","")
    old_assets = elements[3].split(u'\xa0')[0].replace(",","")
    increase = elements[4].split(u'\xa0')[0].replace(",","")
    try:
        increase_pct = float(elements[5].replace('%',''))
    except ValueError:
        increase_pct = elements[5].replace('%','')
    list_repeat = [el,id2,id1,name,new_assets,old_assets,increase,increase_pct]
    titles = ['election','new_id','old_id','name2','assets new','assets old','increase','pct change']
    dict_repeat = dict(zip(titles,list_repeat))
    return dict_repeat

In [316]:
df_repeat = pd.DataFrame()

In [317]:
%%time
for el in list_elections:
    link_repeat = "http://myneta.info/"+el+"/index.php?action=recontestAssetsComparison"
    page = requests.get(link_repeat)
    tree = BeautifulSoup(page.text,"html.parser")
    tables = tree.findAll("table")
    all_lines = tables[2].findAll("tr")[2:]
    info_repeat = [comparison_clean(l,el) for l in all_lines if len(l)==7]
    df_state = pd.DataFrame(info_repeat)
    df_repeat = df_repeat.append(df_state)


Wall time: 22.8 s


In [324]:
#df_repeat.reset_index(inplace=True)
df_repeat.head()

assets new assets old    election    increase  \
0  7950241448   70312891  andhra2014  7879928557   
1  1774895611  686738373  andhra2014  1088157238   
2   631419706   42828523  andhra2014   588591183   
3   218291595   19549518  andhra2014   198742077   
4   386256000  202728660  andhra2014   183527340   

                                 name2 new_id old_id pct change  
0                    Galla Arunakumari   2104   2174      11207  
1              Nara Chandra Babu Naidu   2150   2415        158  
2                  P Ramachandra Reddy   2236   2163       1374  
3  Venkata Sujay Krishna Rangarao Ravu   3959   1634       1017  
4      Dwarampudi Chandrasekhara Reddy   3139   2699         91

In [222]:
for el,list_id in winners_dict.items():
    df_fin['win'][df_fin.election==el]= df_fin.apply(lambda row: (row['election']==el)==True and (row['id'] in list_id)==True, axis=1)

C:\Users\mkkes_000\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [223]:
df_fin.to_csv("tempdata\df_all_regional_elections.csv")

In [229]:
print df_fin.shape
print len(df_fin.consistuency.unique())
print len(df_fin.district.unique())
print len(df_fin.election.unique())
print df_fin[df_fin.win==1].groupby('election')['Gender'].count(), df_fin.groupby('election')['c'].agg( pd.Series.nunique)

df_fin[df_fin.win==1].head()

(37340, 21)
3528
566
25
election
andhra2014          159
arunachal2014        54
bihar2015           234
chhattisgarh2013     87
delhi2015            62
gujarat2012         177
haryana2014          86
hp2012               63
jharkhand2014        76
jk2014               84
karnataka2013       212
kerala2011          134
maharashtra2014     276
manipur2012          57
mp2013              225
odisha2014          140
pb2012              113
rajasthan2013       192
sikkim2014           24
tamilnadu2011       227
telangana2014       107
tripura2013          46
up2012              395
utt2012              67
westbengal2011      285
Name: Gender, dtype: int64 election
andhra2014          160
arunachal2014        54
bihar2015           234
chhattisgarh2013     87
delhi2015            62
gujarat2012         177
haryana2014          86
hp2012               64
jharkhand2014        76
jk2014               84
karnataka2013       219
kerala2011          137
maharashtra2014     281
manipur2012        

Age Criminal Cases Education Level Gender  \
22   54             No        Graduate      M   
26   47            Yes       12th Pass      M   
52   62             No        Graduate      M   
61   53            Yes        Graduate      M   
70   47            Yes       12th Pass      M   

                                         IPC Details*  Immovable Assets  \
22                                                 []          72110000   
26  [Voluntarily causing hurt by dangerous weapons...         443449750   
52                                                 []          74327550   
61  [Disobedience to order duly promulgated by pub...        1547600000   
70  [Acts done by several persons in furtherance o...          18227683   

    Movable Assets                         Name  Number of Cases  \
22        12190851  Vykuntam Prabhakar Chowdary                0   
26        34526763     Gonuguntla Suryanarayana               10   
52         2900069             R.Jithendra Goud                0   
61      2190588692        Balakrishna Nandamuri                1   
70        10343024            Attar Chand Basha                4   

   PAN Given(Y or N)  ...  Serious IPC Counts  Total Assets  \
22                 Y  ...                   0      84300851   
26                 Y  ...                   8     477976513   
52                 Y  ...                   0      77227619   
61                 Y  ...                   0    3738188692   
70                 Y  ...                   0      28570707   

    Total Liabilities    c     consistuency   district    election    id  \
22            9169224  282  Anantapur Urban  Anantapur  andhra2014  1884   
26            5166861  289      Dharmavaram  Anantapur  andhra2014  1904   
52             160000  279         Guntakal  Anantapur  andhra2014  1931   
61          536508006  286         Hindupur  Anantapur  andhra2014  1941   
70            2850604  290           Kadiri  Anantapur  andhra2014  1954   

                                                 link   win  
22  http://myneta.info/andhra2014/candidate.php?ca...  True  
26  http://myneta.info/andhra2014/candidate.php?ca...  True  
52  http://myneta.info/andhra2014/candidate.php?ca...  True  
61  http://myneta.info/andhra2014/candidate.php?ca...  True  
70  http://myneta.info/andhra2014/candidate.php?ca...  True  

[5 rows x 21 columns]

In [331]:
df_fin2 = pd.merge(left = df_fin, right = df_repeat,how="left",left_on = ['election','id'],right_on = ['election','new_id'])

In [332]:
df_fin2.shape

(37340, 28)

In [346]:
df_fin2.to_csv("tempdata\df_all_regional_elections2.csv")